In [ ]:
pip install pandas numpy matplotlib numpy seaborn statsmodels scipy

***0.Import-package***

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize

***import dataset***

In [ ]:
file_path =r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\FINC503\A1 material-python\DATA for CLASS\NZDATA-FORQ1.csv'
df = pd.read_csv(file_path)
print(df)



**Show top 5 rows**

In [ ]:
df.head(5)

**Drop Missing**

***2. data cleaing**

In [ ]:
df=df.dropna()
df.head(5)

**2.1-data cleaning-convert Variable to numeric**

In [ ]:
#df['RET'] = pd.to_numeric(df['RET'], errors='coerce')
df['monly_UNADJUSTED PRICE'] = pd.to_numeric(df['monly_UNADJUSTED PRICE'], errors='coerce')
#keep price >0, ensuring we have correct data
df = df[df['monly_UNADJUSTED PRICE'] > 0]



#winsorized
price_series = df['monly_UNADJUSTED PRICE']
winsorized_price = winsorize(price_series, limits=[0.01, 0.01])
df['monly_UNADJUSTED PRICE'] = winsorized_price


***3. calculate simple return***

In [ ]:
df['Simple_Return'] = df.groupby('Stock_Name')['monly_UNADJUSTED PRICE'].pct_change()



df.head(5)

***4.mean value for simple return ***

In [ ]:
print(df['Simple_Return'].mean())

*** 5.Standatard Deviation for simple return ***

In [ ]:
print(df['Simple_Return'].std())

***5. time identify and Create YYYY-MM***

In [ ]:
df['date']=pd.to_datetime(df['Date'])
df['month']=df['date'].dt.to_period('M')
df.head(5)

**6.first order autocrrelation for simple return***

In [ ]:
lagauto_simple_return= df['Simple_Return'].autocorr(lag=1)
#df[''] is creating a new column
print(lagauto_simple_return)


***7.calcluate continously compund reutrn***

In [ ]:
df['CCR']=np.log1p(df['Simple_Return'])  #log return=ln(1+Simple Return)
print(df['CCR'].std())
print(df['CCR'].mean())


#first order autocrrelation for continiously return
lag_CCR= df['CCR'].autocorr(lag=1)
print(lag_CCR)


In [ ]:
df.describe()

**Create market average monthly mean simple return**
we aggregate simple reutn by the month, and calcualte monthly market mean simple return

In [ ]:
monthlyAvg_Simple = df.groupby('month')['Simple_Return'].mean().reset_index() #return a new dataframe, inlcuding each month's simple return


print(monthlyAvg_Simple)
# groupby()...mean() create a new dataframe. if you did not set 'reset_index() ', pandas porbably automatically set month as index. index cannot be access with normally way,like groupby
#therefore, we set it as not index

**Create market average monthly mean compunded return**
we aggregate simple reutn by the month, and calcualte monthly market mean compunded return

In [ ]:
monthlyAvg_CCR = df.groupby('month')['CCR'].mean().reset_index() #return a new dataframe, inlcuding each month's compunded return
print(monthlyAvg_CCR)
#monthlyAvg_CCR.head(5)

***⭐Create 4-year Groups- calculate each perido's Simple return mean***
first step, give each 4-year group a label.
2000-2003, group 1
2004-2007, group 2
...

***Calculate each 4-year group’s mean simple reutrn***

In [ ]:

monthlyAvg_Simple['year'] = monthlyAvg_Simple['month'].dt.year
monthlyAvg_Simple['year'] = monthlyAvg_Simple['month'].astype(str).str[:4].astype(int)

#create a year group. we use each year mins the first year (ie.2000), and then we integer division by 4 (//4), thewn we get group
#eg, year 2023, will be divdid into group (2023-2000)//4=5. group numbrt=5
monthlyAvg_Simple['year_group'] = ((monthlyAvg_Simple['year'] - monthlyAvg_Simple['year'].min())//4)

#monthlyAvg_Simple.head(5)
#monthlyAvg_Simple.tail(5)
print(monthlyAvg_Simple)

In [ ]:
avg_by_group = monthlyAvg_Simple.groupby('year_group')['Simple_Return'].mean().reset_index()
avg_by_group.rename(columns={'Simple_Return': 'simple_return_mean'}, inplace=True)
print(avg_by_group)

**Create year Range for each group**
add excat year period to the year_group

In [ ]:
base_year = 2000  # start year
avg_by_group['Year_Range'] = (
    (base_year + avg_by_group['year_group'] * 4).astype(str)
    + '-' +
    (base_year + avg_by_group['year_group'] * 4 + 3).astype(str)
)
# eg. for year_group 5, it would be (2000+5*3) to (2000+5*3+2) ------>2015-2017

print(avg_by_group)

**Standard Deviation each year group** for simple return

In [ ]:
std_by_group = monthlyAvg_Simple.groupby('year_group')['Simple_Return'].std().reset_index()
std_by_group.rename(columns={'Simple_Return': 'simple_returnSD'}, inplace=True)
base_year = 2000  # start year
std_by_group['Year_Range'] = (
    (base_year + std_by_group['year_group'] * 4).astype(str)
    + '-' +
    (base_year + std_by_group['year_group'] * 4 + 3).astype(str)
)

print(std_by_group)

**first order autocorrelation** for simple return

In [ ]:
def auto_1(x):
    return x.autocorr(lag=1)
autocorr_by_group = monthlyAvg_Simple.groupby('year_group')['Simple_Return'].apply(auto_1).reset_index()
autocorr_by_group.rename(columns={'Simple_Return': 'simple_return_AC1'}, inplace=True)  ##inplace=true means that change the neme in the dataframe, not in a new dataframe

base_year = 2000  # start year
autocorr_by_group['Year_Range'] = (
    (base_year + autocorr_by_group['year_group'] * 4).astype(str)
    + '-' +
    (base_year + autocorr_by_group['year_group'] * 4 + 3).astype(str)
)

print(autocorr_by_group)

***⭐Create 4-year Groups- calculate each perido's continiously compund return return mean***
first step, give each 4-year group a label.
2000-2003, group 1
2004-2007, group 2
...

In [ ]:

monthlyAvg_CCR['year'] = monthlyAvg_CCR['month'].dt.year
monthlyAvg_CCR['year'] = monthlyAvg_CCR['month'].astype(str).str[:4].astype(int)



#create a year group. we use each year mins the first year (ie.2000), and then we integer division by 4 (//4), thewn we get group
#eg, year 2023, will be divdid into group (2023-2000)//4=5. group numbrt=5
monthlyAvg_CCR['year_group'] = ((monthlyAvg_CCR['year'] - monthlyAvg_CCR['year'].min())//4)

#monthlyAvg_Simple.head(5)
#monthlyAvg_Simple.tail(5)
print(monthlyAvg_CCR)

In [ ]:
avg_by_group = monthlyAvg_CCR.groupby('year_group')['CCR'].mean().reset_index()
avg_by_group.rename(columns={'CCR': 'CCR_MEAN'}, inplace=True)
print(avg_by_group)

**continously compound return each year group** for continious compund reutrn

In [ ]:
base_year = 2000  # start year
avg_by_group['Year_Range'] = (
    (base_year + avg_by_group['year_group'] * 4).astype(str)
    + '-' +
    (base_year + avg_by_group['year_group'] * 4 + 3).astype(str)
)
# eg. for year_group 5, it would be (2000+5*3) to (2000+5*3+2) ------>2015-2017

print(avg_by_group)

**Standard Deviation each year group** for continious compund reutrn

In [ ]:
std_by_group = monthlyAvg_CCR.groupby('year_group')['CCR'].std().reset_index()
std_by_group.rename(columns={'CCR': 'CCR_SD'}, inplace=True)
base_year = 2000  # start year
std_by_group['Year_Range'] = (
    (base_year + std_by_group['year_group'] * 4).astype(str)
    + '-' +
    (base_year + std_by_group['year_group'] * 4 + 3).astype(str)
)

print(std_by_group)

**first order autocorrelation** for continious compound return

In [ ]:
def auto_1(x):
    return x.autocorr(lag=1)
autocorr_by_group_CCR = monthlyAvg_CCR.groupby('year_group')['CCR'].apply(auto_1).reset_index()
autocorr_by_group_CCR.rename(columns={'CCR': 'CCR_AC1'}, inplace=True)  ##inplace=true means that change the neme in the dataframe, not in a new dataframe

base_year = 2000  # start year
autocorr_by_group_CCR['Year_Range'] = (
    (base_year + autocorr_by_group_CCR['year_group'] * 4).astype(str)
    + '-' +
    (base_year + autocorr_by_group_CCR['year_group'] * 4 + 3).astype(str)
)

print(autocorr_by_group_CCR)

***Summary Statistics***

In [ ]:
#df.describe()
df_summary=df[[ 'Simple_Return', 'CCR', 'lag1autocc']].describe()

df_summary.to_csv(r'C:\Users\yumi7517\OneDrive - University of Otago\Desktop\example code\summarystatistics.csv', index=False)
print(df_summary)